# Анализ товарного ассортимента интернет-магазина товаров для дома и быта

**Цель проекта –** проанализировать ассортимент товаров, определить какие товары входят в основной и дополнительный ассортимент, чтобы грамотно предлагать покупателям дополнительные товары и оптимизировать закупки, для этого:
- Проведем исследовательский анализ данных; 
- Проанализируем торговый ассортимент; 
- Сформулируем и проверим статистические гипотезы.

*В качестве входных данных используется датасет описывающий транзакции интернет-магазина товаров для дома и быта.*

[Презентация проекта](https://disk.yandex.ru/i/T6h0U-2-ByQtLw)  
[Дашборд](https://public.tableau.com/app/profile/ljhl/viz/EcommerceDashboard_16554073462910/Dashboard1)

**Ход анализа:**
1. Обзор данных;
2. Предобработка данных;
3. Категоризация товарного ассортимента;
4. Проведение исследовательского анализа данных (EDA);
5. Проверка статистических гипотез;
6. Расчет бизнес-показателей.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from scipy import stats as st
from pymystem3 import Mystem
from plotly.subplots import make_subplots

In [2]:
%config InlineBackend.figure_format = 'retina'
pd.set_option('display.max_colwidth', 100)
pd.set_option("display.precision", 2)
pio.templates.default = 'seaborn'

In [3]:
df = pd.read_csv('ecommerce_dataset.csv')

## Обзор данных

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6737 entries, 0 to 6736
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         6737 non-null   int64  
 1   customer_id  6737 non-null   object 
 2   order_id     6737 non-null   int64  
 3   product      6737 non-null   object 
 4   quantity     6737 non-null   int64  
 5   price        6737 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 315.9+ KB


**Согласно документации к данным:**  

- `date` — дата заказа;  
- `customer_id` — идентификатор покупателя; order_id — идентификатор заказа;  
- `product` — наименование товара;  
- `quantity` — количество товара в заказе; price — цена товара.

In [5]:
df.sample(5)

,date,customer_id,order_id,product,quantity,price
721,2018110908,8dbfb5d0-837c-4cb7-a5d9-88ff7ed304e8,68919,Муляж Морковь 16 см,1,59.0
4968,2019061007,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,"Рассада Кабачка сорт Аэронавт, кассета по 6шт",1,120.0
3525,2019051119,dddcafaf-6ca9-4427-9e54-a1cdd9323bec,14750,Рассада Арбуза сорт Огонек горшок 9х9 см P-9,1,38.0
4730,2019060616,6a86cc77-ef15-496f-b5d3-89005597ee5d,14856,Бадан Сердцелистный Красная звезда красный объем 1 л,1,150.0
3798,2019051508,3976660e-6cca-4009-a170-be13f13ed459,14778,Арбуз Волгоградец Р-9,1,38.0


In [6]:
df.duplicated().sum()

0

### Вывод

- В датасете нет пропусков;
- В датасете нет явных дубликатов;
- Дата представлена с неправильным типом данных;
- Столбец price можно преобразовать к int.

## Предобработка данных

**Чтобы предобработать данные, выполним следующие действия:**

- Преобразуем типы данных;
- Заменим "ё" на "е";
- Произведем поиск неявных дубликатов;
- Добавим столбцы необходимые для дальнейшего исследования.
  

### Преобразование типов данных

In [7]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d%H')

In [8]:
df['price'] = df['price'].astype('int')

**Заменим "ё" на "е":**

In [9]:
df['product'] = df['product'].str.replace('ё', 'е', regex=True).str.replace('Ë', 'Е', regex=True)

### Поиск неявных дубликатов

In [10]:
df.duplicated(subset=['customer_id', 'order_id', 'product', 'quantity', 'price']).sum()

1864

In [11]:
df[df.duplicated(subset=['customer_id', 'order_id', 'product', 'quantity', 'price'], keep=False)].head(20)

,date,customer_id,order_id,product,quantity,price
15,2018-10-01 18:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,"Мини-сковорода Marmiton ""Сердце"" с антипригарным покрытием 12 см, LG17085",1,239
16,2018-10-01 18:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,Сковорода алюминиевая с антипригарным покрытием MARBLE ALPENKOK d = 26 см AK-0039A/26N,1,824
17,2018-10-01 18:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,Стеклянная крышка для сковороды ALPENKOK 26 см AK-26GL,1,262
18,2018-10-01 18:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,"Сушилка для белья напольная Colombo Star 18, 3679",1,1049
19,2018-10-01 21:00:00,b731df05-98fa-4610-8496-716ec530a02c,68474,Доска гладильная Eurogold Professional 130х48 см металлическая сетка 35748W,1,3299
58,2018-10-02 18:00:00,b731df05-98fa-4610-8496-716ec530a02c,68474,Доска гладильная Eurogold Professional 130х48 см металлическая сетка 35748W,1,3299
59,2018-10-02 19:00:00,b731df05-98fa-4610-8496-716ec530a02c,68474,Доска гладильная Eurogold Professional 130х48 см металлическая сетка 35748W,1,3299
60,2018-10-02 20:00:00,b731df05-98fa-4610-8496-716ec530a02c,68474,Доска гладильная Eurogold Professional 130х48 см металлическая сетка 35748W,1,3299
63,2018-10-03 04:00:00,b731df05-98fa-4610-8496-716ec530a02c,68474,Доска гладильная Eurogold Professional 130х48 см металлическая сетка 35748W,1,3299
80,2018-10-04 09:00:00,32de7df8-8d4f-4c84-a7b9-c41d00dd83ba,68522,Эвкалипт Гунни d-17 см h-60 см,1,1409


In [12]:
round(df.duplicated(subset=['customer_id', 'order_id', 'product', 'quantity', 'price']).sum() * 100 / len(df), 2)

27.67

*Имеется почти 28% неявных дубликатов.*

**Посмотрим на некоторые заказы:**

In [13]:
df[df['order_id'] == 68476]

,date,customer_id,order_id,product,quantity,price
15,2018-10-01 18:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,"Мини-сковорода Marmiton ""Сердце"" с антипригарным покрытием 12 см, LG17085",1,239
16,2018-10-01 18:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,Сковорода алюминиевая с антипригарным покрытием MARBLE ALPENKOK d = 26 см AK-0039A/26N,1,824
17,2018-10-01 18:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,Стеклянная крышка для сковороды ALPENKOK 26 см AK-26GL,1,262
18,2018-10-01 18:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,"Сушилка для белья напольная Colombo Star 18, 3679",1,1049
273,2018-10-16 17:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,"Мини-сковорода Marmiton ""Сердце"" с антипригарным покрытием 12 см, LG17085",1,239
274,2018-10-16 17:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,Сковорода алюминиевая с антипригарным покрытием MARBLE ALPENKOK d = 26 см AK-0039A/26N,1,824
275,2018-10-16 17:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,Стеклянная крышка для сковороды ALPENKOK 26 см AK-26GL,1,262
276,2018-10-16 17:00:00,17213b88-1514-47a4-b8aa-ce51378ab34e,68476,"Сушилка для белья напольная Colombo Star 18, 3679",1,1049


In [14]:
df[df['order_id'] == 68574]

,date,customer_id,order_id,product,quantity,price
140,2018-10-08 15:00:00,3de09660-90bc-4a28-aaf1-34c8435fe59c,68574,"Таз пластмассовый 15,0 л пищевой овальный ""Ekko"" 2775, 1404032",1,209
141,2018-10-08 15:00:00,3de09660-90bc-4a28-aaf1-34c8435fe59c,68574,"Таз пластмассовый 18,0 л пищевой (Иж), 1404047",1,194
142,2018-10-08 15:00:00,3de09660-90bc-4a28-aaf1-34c8435fe59c,68574,"Таз пластмассовый 20,0 л пищевой (Минеральные воды), 1404045",1,277
143,2018-10-08 15:00:00,3de09660-90bc-4a28-aaf1-34c8435fe59c,68574,"Таз пластмассовый 24,0 л пищевой круглый (Иж), 1404006",1,239
147,2018-10-08 19:00:00,3de09660-90bc-4a28-aaf1-34c8435fe59c,68574,"Таз пластмассовый 15,0 л пищевой овальный ""Ekko"" 2775, 1404032",1,209
148,2018-10-08 19:00:00,3de09660-90bc-4a28-aaf1-34c8435fe59c,68574,"Таз пластмассовый 18,0 л пищевой (Иж), 1404047",1,194
149,2018-10-08 19:00:00,3de09660-90bc-4a28-aaf1-34c8435fe59c,68574,"Таз пластмассовый 20,0 л пищевой (Минеральные воды), 1404045",1,277
150,2018-10-08 19:00:00,3de09660-90bc-4a28-aaf1-34c8435fe59c,68574,"Таз пластмассовый 24,0 л пищевой круглый (Иж), 1404006",1,239
152,2018-10-09 06:00:00,3de09660-90bc-4a28-aaf1-34c8435fe59c,68574,"Таз пластмассовый 15,0 л пищевой овальный ""Ekko"" 2775, 1404032",1,209
153,2018-10-09 06:00:00,3de09660-90bc-4a28-aaf1-34c8435fe59c,68574,"Таз пластмассовый 18,0 л пищевой (Иж), 1404047",1,194


*Скорее всего система учитывает любые действия с заказом, покупатель может добавлять и удалять их с разным промежутком времени, т.к. у нас нет информации был ли оплачен заказ, актуальным целесообразно считать последний заказ.*

In [15]:
df = df.drop_duplicates(subset=['customer_id', 'order_id', 'product', 'quantity', 'price'], keep = 'last')

**Проверим, есть ли у одного заказа несколько покупателей:**

In [16]:

many_customers = (df
            .groupby('order_id')
            .agg({'customer_id': 'nunique'})
            .query('customer_id > 1')
)
many_customers

,customer_id
order_id,
14872,2
68785,2
69283,2
69310,2
69345,2
69410,2
69485,3
69531,2
69833,2


In [17]:
df[df['order_id'] == 72845]

,date,customer_id,order_id,product,quantity,price
6504,2019-10-03 14:00:00,d8465f63-35db-4809-aff3-a8f7ebfc257f,72845,Муляж Яблоко зеленый 9 см полиуретан,40,59
6505,2019-10-03 15:00:00,0309d37c-ab5f-4793-ba72-5484c019b840,72845,Муляж Яблоко зеленый 9 см полиуретан,40,59
6508,2019-10-04 08:00:00,25a8cd52-3efa-48ee-a6bd-d413d7e2b42f,72845,Муляж Яблоко зеленый 9 см полиуретан,40,59
6538,2019-10-07 20:00:00,2ac05362-3ca7-4d19-899c-7ba266902611,72845,Муляж Яблоко зеленый 9 см полиуретан,40,59


*Различие в id покупателя обусловлено тем, что для разных устройств назначается свой id. Будем считать, что последний заказ самый актуальный.*

In [18]:
df = df.drop_duplicates(subset=['order_id', 'product', 'quantity', 'price'], keep = 'last')

### Добавление столбцов для дальнейшего исследования

**Добавим столбец с выручкой:**

In [19]:
df['revenue'] = df['quantity'] * df['price']

**Добавим столбцы с часом, днем, днем недели, месяцем и годом заказа:**

In [20]:
df['hour'] = df['date'].dt.hour
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.day_name() 
df['week'] = df['date'].dt.isocalendar().week 
df['month'] = df['date'].astype('datetime64[M]')
df['year'] = df['date'].dt.year 
df['dt'] = df['date'].astype('datetime64[D]')

**Так же добавим столбцы указывающие, что товар в заказе был основным или дополнительным:**

**Количество уникальных товаров в заказе:**

In [21]:
df['unique_products_count'] = df.groupby('order_id')['product'].transform('nunique') 

**Если товар один в заказе, то 1, нет 0:**

In [22]:
df['main'] = df['unique_products_count'].apply(lambda x: 1 if x == 1 else 0)

**Если товар не один в заказе, то 1, нет 0:**

In [23]:
df['additional'] = df['unique_products_count'].apply(lambda x: 0 if x == 1 else 1)

### Вывод

- Дата была приведена к формату даты;
- Столбец `price` был пирведен к типу `int`;
- Были найдены и удалены неявные дубликаты;
- Добавлены дополнительные столбцы:
  - С часом заказа;
  - С днем заказа;
  - Днем недели заказа;
  - Месяцем заказа;
  - Годом заказа;
  - Столбец с выручкой;
  - Столбец с количеством уникальных товаров в заказе;
  - Столбец указывающий, что товар основной в заказе;
  - Столбец указывающий, что товар дополнительный в заказе;

**Так же хотелось бы отметить, неявные дубликаты – это следствие того, что система учитывает любые действия с заказом, будь то отмена или удаление заказа, в результате этого записи в таблице дублируются. Еще система присваивает разные идентификаторы одному и тому же пользователю для разных устройств.**

## Категоризация товарного ассортимента

**Чтобы категоризировать товар, создадим словарь с категориями, а после прогоним наименования товаром через лемматизацию.**

**Словарь получен, следующим образом:**

1. Датафрейм сортируется по количеству проданного товара по убыванию;
2. Выделяются ключевые слова и заносятся в категории к которым они больше всего подходят;
3. Все что не категоризировалось попадает в категорию "Прочее";
4. Датафрейм снова сортируется по количеству проданного товара, но только по категории "Прочее";
5. Повторяем шаг "1" пока товарный ассортимент не попадет в нужные категории.

In [24]:
categories = { 
    'Кухонные принадлежности': ['тарелка','венчик', 'косточка', 'вилка', 'сковорода', 'рассекатель', 'лопатка',
                                'скалка', 'терка', 'толкушка', 'блюдо', 'термос', 'сотейник', 'рыбочистка', 'пароварка',
                                'ложка', 'кружка', 'половник', 'Luminarc', 'тряпкодержатель', 'орехоколка', 'соковарка',
                                'нож', 'скатерть', 'котел', 'термокружок', 'противень', 'сахарница', 'пресс',
                                'салфетница', 'столовый','сервировочный', 'заварочный', 'трудновыводимый', 'соковыжималка',
                                'хлебница', 'кухня', 'кастрюля', 'посуда', 'кувшин', 'просеиватель', 'стакан',
                                'тортница', 'контейнер', 'овощечистка', 'банка', 'миска', 'кухонный', 'разделочный',
                                'свч', 'банка', 'заварочный', 'салатник', 'дуршлаг', 'мерный', 'лоток', 'губка',
                                'термостакан', 'WEBBER', 'фужер', 'ножеточка', 'выпечка', 'картофелемялка',
                                'чайник', 'электроштопор', 'мантоварка', 'миксер', 'овощеварка', 'настольный'
                               ],  

    'Сад и огород': ['пеларгония','бархатцы','сельдерей','суккулент','рассада','клен','бакоп','юкка','шеффлер', 'лузеан','растение', 
                    'алоэ','радермахер','хризолидокарпус', 'лутесценс','томата','пеларгония', 'роза','петуния','герань', 'цветок',
                    'однолетнее','флокс','цикламен', 'примула','калибрахоа','фуксия','вербена','пуансеттия','фиалка','дыня',
                    'пиретрум', 'гайлардий', 'горох', 'любисток', 'змееголовник', 'валериана', 'анемон', 'лаватер', 'физостегия',
                    'ель', 'лук', 'шалфей', 'георгин', 'лилейник', 'платикодон', 'энотера', 'годеция', 'эшшольций', 'эхинацея',
                    'солидаго', 'бузульник', 'смолевка', 'незабудка', 'кодонант', 'морковь', 'гиностемма',
                    'комнатное','базилик','бегония','бальзамин','бакопа','космея','мята','антуриум','огурец','хризантема', 'лапчатка',
                    'эвкалипт','декабрист','томат','гвоздика','арбуз','петрушка','цинния', 'патиссон','алиссум','азалия','тимьян',
                    'лобелия', 'капуста', 'газания', 'циперус','виола', 'хлорофитум', 'лаванда', 'розмарин', 'гипсофила', 'гвоздик',
                    'мимоза', 'мединилла', 'тагетис', 'земляника', 'астра', ' зверобой', 'настурция', 'папоротник','календула',
                    'каланхое', 'тыква', 'гербера','цветущее', 'афеляндра', 'нивянник', 'вербейник', 'гардения','d', 'гортензия',
                    'калатея', 'алое', 'кореопсис', 'укроп', 'вигна', 'скиммия', 'колеус', 'душица', 'фатсия', 'лантана', 'кабачок',
                    'салат', 'осина', 'целозия', 'портулак', 'крассула', 'аргирантерум', 'хоста', 'цинерария', 'монарда', 'тюльпан',
                    'баклажан', 'вероника', 'сальвия', 'кориандр', 'лен', 'цитрофортунелла', 'пахира', 'фаленопсис', 'бадан',
                    'эхеверия', 'клубника', 'многолетнее', 'кофе', 'седум', 'табак', 'спатифиллум', 'ранункулус', 'барвинок',
                    'дендориум', 'калла', 'лавр', 'мирт','львиный', 'дендробиум', 'цветочная', 'кипарисовик', 'аврора', 'рудбекия',
                    'колокольчик', 'овсянница', 'камнеломка', 'котовник', 'ясколка', 'd', 'h', 'кашпо', 'аквилегия', 'косметь'
                    ], 

    'Товары для дома': ['сушилка', 'одежда', 'костюм', 'плечики', 'вешалка', 'прищепок', 'посудомоечный', 'бидон', 'сменный',
                        'кофр', 'корзина', 'корзинка', 'подставка', 'шпагат', 'коробка', 'почтовый', 'коврик',
                        'хранение', 'бак', 'подкладка', 'чехол', 'гладильный', 'подрукавник', 'фиксатор', 'глажение',
                        'урна', 'придверный', 'термометр', 'влаговпитывающий', 'скребок', 'ткань', 'зажигалка',
                        'таз', 'ведро', 'корыто', 'ковш', 'колесо', 'сумка', 'тележка', 'стеллажный', 'шнур', 'пьезозажигалка',
                        'простыня', 'одеяло', 'наматрацник','пододеяльник', 'наволочка', 'полотенце', 'покрывало', 'наматрасник',
                        'плед', 'наматрицник', 'белье', 'подушка', 'напольный', 'утюг', 'фен', 'пылесос', 'туалетный', 'увлажнять',
                        'ванный', 'халат', 'ванна', 'зубной', 'противоскользящий', 'мыло', 'унитаз', 'подголовник'
                       ],

    'Интереьер': ['искусственный', 'искуственный', 'цветок', 'пуф','комод', 'фоторамка',
                  'интерьерный', 'муляж', 'декоративный', 'светильник', 'штора', 'стеллаж',
                  'обувница', 'этажерка', 'ключница'
                 ],                   

    'Все для ремонта': ['стремянка', 'скоба', 'пружина', 'крепеж', 'уголок', 'карниз'
                        'завертка', 'шпингалет', 'крючок', 'петля', 'стяжка', 'угольник',
                        'инструмент', 'полка', 'полк', 'фал', 'напильник', 'насадка валик',
                        'нитрид', 'сварка', 'засор', 'дерево', 'электрический', 'завертка',
                        'штангенциркуль', 'мебельный', 'линейка', 'гои'
                       ],

    'Все для уборки': ['веник', 'совок', 'швабра', 'мусорный', 'мусор', 'салфетка', 'отбеливатель', 'биопорошок',
                       'щетка', 'ерш', 'вантуз', 'перчатка', 'сметка', 'мусор', 'DECS', 'ROZENBAL',
                       'окномойка', 'тряпка', 'микрофибры', 'мытье', 'вентиляционный', 'антижир', 'стирка'
                      ],  
}

In [25]:
def create_category(col, categories, instance):
    """
    Функция для создания категорий путем лемматизации строки.

    row - столбец датафрейма
    category - категории товаров
    instance - экземпляр класса для лемматизации
    """

    lemmas = ' '.join(instance.lemmatize(col)).split()
    for lemma in lemmas:
        for key, values in categories.items():
            if lemma in values:
                return key
    return 'Прочее'

In [26]:
m = Mystem()

In [27]:
df['category'] = df['product'].apply(create_category, categories=categories, instance=m)

In [28]:
df['category'].value_counts()

Сад и огород               2519
Товары для дома            1336
Кухонные принадлежности     382
Интереьер                   342
Все для ремонта             131
Все для уборки              130
Name: category, dtype: int64

*Основной товарной категорией является "Сад и огород".*

## Исследовательский анализ данных (EDA)

**Чтобы провести исследовательский анализ данных, выполним следующие действия:**
- Проверим описательную статистику;
- Проанализируем:
  - Количество заказов;
  - Количество заказов по времени;
  - Количество заказов по сезонам с учетом категории товаров;
  - Количество покупателей по времени;
  - Выручку по времени;
  - Средний чек по времени;
  - Товарный ассортимент;
  - Основной и дополнительный товар.

### Проверка описательной статистики

In [29]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_id,4840.0,48110.61,27362.54,12624.0,14773.25,68854.0,70812.5,73164.0
quantity,4840.0,2.82,17.64,1.0,1.00,1.0,1.0,1000.0
price,4840.0,514.93,945.45,9.0,90.00,150.0,488.0,14917.0
revenue,4840.0,850.55,9840.40,9.0,120.00,194.0,734.0,675000.0
hour,4840.0,13.72,4.80,0.0,10.00,13.0,17.0,23.0
day,4840.0,15.39,8.80,1.0,8.00,15.0,23.0,31.0
week,4840.0,26.62,14.25,1.0,16.00,23.0,41.0,52.0
year,4840.0,2018.75,0.43,2018.0,2018.00,2019.0,2019.0,2019.0
unique_products_count,4840.0,6.25,8.76,1.0,1.00,2.0,8.0,51.0
main,4840.0,0.49,0.50,0.0,0.00,0.0,1.0,1.0


- Среднее количество товаров в заказе 2.8;
- Имеются очень крупные заказы в 1000 штук;
- Большой разброс цен, стандартное отклонение 945;
- В магазине есть те, кто закупается на очень крупные суммы;
- Медиана времени закупки приходится на 13 часов;
- Поровну распределились основные и дополнительные товары.

### Количество заказов

In [30]:
df.sort_values(by='quantity', ascending=False).head(10)

,date,customer_id,order_id,product,quantity,price,revenue,hour,day,day_of_week,week,month,year,dt,unique_products_count,main,additional,category
5456,2019-06-18 15:00:00,312e9a3e-5fca-43ff-a6a1-892d2b2d5ba6,71743,"Вантуз с деревянной ручкой d14 см красный, Burstenmann, 0522/0000",1000,675,675000,15,18,Tuesday,25,2019-06-01,2019,2019-06-18,1,1,0,Все для уборки
5071,2019-06-11 07:00:00,146cd9bf-a95c-4afb-915b-5f6684b17444,71668,Вешалки мягкие для деликатных вещей 3 шт шоколад,334,148,49432,7,11,Tuesday,24,2019-06-01,2019,2019-06-11,1,1,0,Товары для дома
3961,2019-05-20 21:00:00,5d189e88-d4d6-4eac-ab43-fa65a3c4d106,71478,Муляж ЯБЛОКО 9 см красное,300,51,15300,21,20,Monday,21,2019-05-01,2019,2019-05-20,1,1,0,Интереьер
1158,2018-12-10 14:00:00,a984c5b7-ff7e-4647-b84e-ef0b85a2762d,69289,"Ручка-скоба РС-100 белая *Трибатрон*, 1108035",200,29,5800,14,10,Monday,50,2018-12-01,2018,2018-12-10,1,1,0,Все для ремонта
568,2018-11-01 08:00:00,aa42dc38-780f-4b50-9a65-83b6fa64e766,68815,Муляж ЯБЛОКО 9 см красное,170,51,8670,8,1,Thursday,44,2018-11-01,2018,2018-11-01,1,1,0,Интереьер
267,2018-10-16 08:00:00,cd09ea73-d9ce-48c3-b4c5-018113735e80,68611,"Пружина дверная 240 мм оцинкованная (Д-19 мм) без крепления, 1107014",150,38,5700,8,16,Tuesday,42,2018-10-01,2018,2018-10-16,2,0,1,Все для ремонта
2431,2019-03-23 10:00:00,685d3d84-aebb-485b-8e59-344b3df8b3d3,70841,Плечики пластмассовые Размер 52 - 54 Тула 1205158,150,20,3000,10,23,Saturday,12,2019-03-01,2019,2019-03-23,1,1,0,Товары для дома
266,2018-10-16 08:00:00,cd09ea73-d9ce-48c3-b4c5-018113735e80,68611,"Крепеж для пружины дверной, 1107055",150,19,2850,8,16,Tuesday,42,2018-10-01,2018,2018-10-16,2,0,1,Все для ремонта
586,2018-11-02 11:00:00,0c5aaa88-e346-4f87-8f7a-ad8cbc04e965,68831,Муляж ЯБЛОКО 9 см красное,140,59,8260,11,2,Friday,44,2018-11-01,2018,2018-11-02,1,1,0,Интереьер
1103,2018-12-04 17:00:00,7d255526-fcc2-4f79-b28a-217d7d2373a8,69206,"Щетка для посуды *ОЛЯ*, Мультипласт 1807010",100,26,2600,17,4,Tuesday,49,2018-12-01,2018,2018-12-04,1,1,0,Все для уборки


*Есть крупные оптовые заказы.*

**Посмотрим много ли таких заказов:**

In [31]:
np.percentile(df['quantity'], [90, 95, 99])

array([ 3.,  7., 30.])

**За точку отсчета возьмем 99 перцентиль:**

In [32]:
very_large_orders = df[df['quantity'] > 30]

In [33]:
very_large_orders['order_id'].nunique()

36

**Посмотрим, как часто данные продукты заказывали:**

In [34]:
very_large_orders_counts = (df[df['product'].isin(very_large_orders['product'].unique())]
                            .groupby('product')
                            .agg({'order_id': 'nunique'})
                            .sort_values(by='order_id', ascending=False) 
                            .reset_index()
)
very_large_orders_counts

,product,order_id
0,Муляж Яблоко зеленый 9 см полиуретан,7
1,Муляж Банан желтый 21 см полиуретан,6
2,Муляж ЯБЛОКО 9 см красное,6
3,"Стяжка оконная с болтом СТ-55 цинк, 1108354",5
4,Муляж Лимон желтый 9 см полиуретан,5
5,Салфетка Protec Textil Polyline 30х43 см Аметист белая 6230,4
6,Цветок искусственный Гвоздика тканевая красная 50 см,3
7,Тележка багажная DELTA ТБР-22 синий грузоподъемность 20 кг сумка и 50 кг каркас РОССИЯ,3
8,Плечики пластмассовые Размер 52 - 54 Тула 1205158,2
9,Вешалки мягкие для деликатных вещей 3 шт шоколад,2


**Оставим только те, которые заказывали больше 1 раза:**

In [35]:
for_drop = very_large_orders_counts[very_large_orders_counts['order_id'] == 1]['product']

In [36]:
df = df.drop(df[df['product'].isin(for_drop)].index)

*Были очень крупные заказы, оставили только те, в которых товар покупался не единожды.*

### Количество заказов по времени

**Выясним за какой промежуток времени мы обладаем данными:**

In [37]:
min_date = df['date'].min()
min_date

Timestamp('2018-10-01 00:00:00')

In [38]:
max_date = df['date'].max()
max_date

Timestamp('2019-10-31 16:00:00')

*Мы обладаем данными с 1 октября 2018 по 31 октября 2019.*

**Посмотрим на распределение заказов:**

In [39]:
cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
orders_pivot_by_month = df.pivot_table(index='month', values='order_id', aggfunc='nunique').reset_index()
orders_pivot_by_day = df.pivot_table(index='day', values='order_id', aggfunc='nunique').reset_index()
orders_pivot_by_weekday = df.pivot_table(index='day_of_week', values='order_id', aggfunc='nunique').reindex(cats).reset_index()
orders_pivot_by_hour = df.pivot_table(index='hour', values='order_id', aggfunc='nunique').reset_index()

In [40]:
def plot_bar_nm(n_rows, m_cols, params):
    """
    Функция для построения столбчатых диаграмм размерности n x m.

    n_rows - количество строк
    m_cols - количество столбцов
    params - параметры для построения
    """

    fig = make_subplots(rows=n_rows, cols=m_cols, horizontal_spacing = 0.05)

    for key, value in params.items():
        data = value['df']
        fig.add_trace(go.Bar(
            name=value['name'],
            x=data[value['x']],
            y=data[value['y']],
            hoverinfo=value['hoverinfo'],),
            row=value['row'], col=value['col']
            )

    return fig

In [41]:
orders_params = {
    'data1': {'df': orders_pivot_by_month, 'name': 'По месяцам', 'x': 'month', 'y': 'order_id', 'hoverinfo': 'y', 'row': 1, 'col': 1},
    'data2': {'df': orders_pivot_by_day, 'name': 'По дням', 'x': 'day', 'y': 'order_id', 'hoverinfo': 'y', 'row': 1, 'col': 2},
    'data3': {'df': orders_pivot_by_weekday, 'name': 'По дням недели', 'x': 'day_of_week', 'y': 'order_id', 'hoverinfo': 'y', 'row': 2, 'col': 1},
    'data4': {'df': orders_pivot_by_hour, 'name': 'По часам', 'x': 'hour', 'y': 'order_id', 'hoverinfo': 'y', 'row': 2, 'col': 2},
}

In [42]:
fig = plot_bar_nm(2, 2, orders_params)

fig.update_xaxes(tickangle=45, tickvals = orders_pivot_by_month['month'], showgrid=False, row=1, col=1)
fig.update_xaxes(tickangle=45, tickvals = orders_pivot_by_day['day'], showgrid=False, row=1, col=2)
fig.update_xaxes(showgrid=False, row=2, col=1)
fig.update_xaxes(tickangle=45, tickvals = orders_pivot_by_hour['hour'], showgrid=False, row=2, col=2)

fig.update_layout(title_text='Количество заказов по времени', height=800)
fig.show()

- Больше всего заказов с февраля по март включительно, учитывая основной ассортимент "Сад и огород", люди готовятся к дачному сезону;
- По дням картина плюс минус одинаковая;
- Больше всего заказов делают в будние дни, меньше всего по выходным;
- Пик заказов приходится на промежуток с 10 до 13 часов.

### Количество заказов по сезонам с учетом категории товаров

**Разобьем датафрейм на 4 сезона:**

In [43]:
winter = (df[df['month']
            .dt
            .strftime('%Y-%m-%d')
            .str
            .contains(r'\d{4}\-12\-01|\d{4}\-01\-01|\d{4}\-02\-01', regex=True)
            ]
)
winter_pivot = (winter
                .pivot_table(index='category', values='order_id', aggfunc='nunique')
                .sort_values(by='order_id', ascending=False)
                .reset_index()
)

In [44]:
spring = (df[df['month']
            .dt
            .strftime('%Y-%m-%d')
            .str
            .contains(r'\d{4}\-03\-01|\d{4}\-04\-01|\d{4}\-05\-01', regex=True)
            ]
)
spring_pivot = (spring
                .pivot_table(index='category', values='order_id', aggfunc='nunique')
                .sort_values(by='order_id', ascending=False)
                .reset_index()
)

In [45]:
summer = (df[df['month']
            .dt
            .strftime('%Y-%m-%d')
            .str
            .contains(r'\d{4}\-06\-01|\d{4}\-07\-01|\d{4}\-08\-01', regex=True)
            ]
)
summer_pivot = (summer
                .pivot_table(index='category', values='order_id', aggfunc='nunique')
                .sort_values(by='order_id', ascending=False)
                .reset_index()
)

In [46]:
autumn = (df[df['month']
            .dt
            .strftime('%Y-%m-%d')
            .str
            .contains(r'\d{4}\-09\-01|\d{4}\-10\-01|\d{4}\-11\-01', regex=True)
            ]
)
autumn_pivot = (autumn
                .pivot_table(index='category', values='order_id', aggfunc='nunique')
                .sort_values(by='order_id', ascending=False)
                .reset_index()
)

In [47]:
seasons_params = {
    'data1': {'df': winter_pivot, 'name': 'Зима', 'x': 'category', 'y': 'order_id', 'hoverinfo': 'y', 'row': 1, 'col': 1},
    'data2': {'df': spring_pivot, 'name': 'Весна', 'x': 'category', 'y': 'order_id', 'hoverinfo': 'y', 'row': 1, 'col': 2},
    'data3': {'df': summer_pivot, 'name': 'Лето', 'x': 'category', 'y': 'order_id', 'hoverinfo': 'y', 'row': 2, 'col': 1},
    'data4': {'df': autumn_pivot, 'name': 'Осень', 'x': 'category', 'y': 'order_id', 'hoverinfo': 'y', 'row': 2, 'col': 2},
}

In [48]:
fig = plot_bar_nm(2, 2, seasons_params)

fig.update_xaxes(tickangle=20, showgrid=False, row=1, col=1)
fig.update_xaxes(tickangle=20, showgrid=False, row=1, col=2)
fig.update_xaxes(tickangle=20, showgrid=False, row=2, col=1)
fig.update_xaxes(tickangle=20, showgrid=False, row=2, col=2)

fig.update_layout(title_text='Количество заказов по сезонам с учетом категории товаров', height=800)
fig.show()

*Весной, ожидаемо, товары категории "Сад и огород" продаются лучше, чем в остальные сезоны.*

### Количество покупателей по времени

In [49]:
customers_pivot_by_month = df.pivot_table(index='month', values='customer_id', aggfunc='nunique').reset_index()
customers_pivot_by_day = df.pivot_table(index='day', values='customer_id', aggfunc='nunique').reset_index()
customers_pivot_by_weekday = df.pivot_table(index='day_of_week', values='customer_id', aggfunc='nunique').reindex(cats).reset_index()
customers_pivot_by_hour = df.pivot_table(index='hour', values='customer_id', aggfunc='nunique').reset_index()

In [50]:
customers_params = {
    'data1': {'df': customers_pivot_by_month, 'name': 'По месяцам', 'x': 'month', 'y': 'customer_id', 'hoverinfo': 'y', 'row': 1, 'col': 1},
    'data2': {'df': customers_pivot_by_day, 'name': 'По дням', 'x': 'day', 'y': 'customer_id', 'hoverinfo': 'y', 'row': 1, 'col': 2},
    'data3': {'df': customers_pivot_by_weekday, 'name': 'По дням недели', 'x': 'day_of_week', 'y': 'customer_id', 'hoverinfo': 'y', 'row': 2, 'col': 1},
    'data4': {'df': customers_pivot_by_hour, 'name': 'По часам', 'x': 'hour', 'y': 'customer_id', 'hoverinfo': 'y', 'row': 2, 'col': 2},
}

In [51]:
fig = plot_bar_nm(2, 2, customers_params)

fig.update_xaxes(tickangle=45, tickvals = customers_pivot_by_month['month'], showgrid=False, row=1, col=1)
fig.update_xaxes(tickangle=45, tickvals = customers_pivot_by_day['day'], showgrid=False, row=1, col=2)
fig.update_xaxes(showgrid=False, row=2, col=1)
fig.update_xaxes(tickangle=45, tickvals = customers_pivot_by_hour['hour'], showgrid=False, row=2, col=2)

fig.update_layout(title_text='Количество покупателей по времени', height=800)
fig.show()

*Картина аналогична количеству заказов*

### Размер выручки по времени

In [52]:
revenue_pivot_by_month = df.pivot_table(index='month', values='revenue', aggfunc='sum').reset_index()
revenue_pivot_by_day = df.pivot_table(index='day', values='revenue', aggfunc='sum').reset_index()
revenue_pivot_by_weekday = df.pivot_table(index='day_of_week', values='revenue', aggfunc='sum').reindex(cats).reset_index()
revenue_pivot_by_hour = df.pivot_table(index='hour', values='revenue', aggfunc='sum').reset_index()

In [53]:
revenue_params = {
    'data1': {'df': revenue_pivot_by_month, 'name': 'По месяцам', 'x': 'month', 'y': 'revenue', 'hoverinfo': 'y', 'row': 1, 'col': 1},
    'data2': {'df': revenue_pivot_by_day, 'name': 'По дням', 'x': 'day', 'y': 'revenue', 'hoverinfo': 'y', 'row': 1, 'col': 2},
    'data3': {'df': revenue_pivot_by_weekday, 'name': 'По дням недели', 'x': 'day_of_week', 'y': 'revenue', 'hoverinfo': 'y', 'row': 2, 'col': 1},
    'data4': {'df': revenue_pivot_by_hour, 'name': 'По часам', 'x': 'hour', 'y': 'revenue', 'hoverinfo': 'y', 'row': 2, 'col': 2},
}

In [54]:
fig = plot_bar_nm(2, 2, revenue_params)

fig.update_xaxes(tickangle=45, tickvals = revenue_pivot_by_month['month'], showgrid=False, row=1, col=1)
fig.update_xaxes(tickangle=45, tickvals = revenue_pivot_by_day['day'], showgrid=False, row=1, col=2)
fig.update_xaxes(showgrid=False, row=2, col=1)
fig.update_xaxes(tickangle=45, tickvals = revenue_pivot_by_hour['hour'], showgrid=False, row=2, col=2)

fig.update_layout(title_text='Размер выручки по времени', height=800)
fig.show()

- Самую большую выручку магазин сделал в последние месяцы 2018 года;
- Во вторник выручка больше, чем в другие дни;
- В течении суток картина аналогична количеству заказов и основная выручка делается днем.

### Средний чек по времени

In [55]:
avg_revenue_pivot_by_month = df.pivot_table(index=['month', 'order_id'], values='revenue').groupby('month').agg({'revenue': 'mean'}).reset_index()
avg_revenue_pivot_by_day = df.pivot_table(index=['day', 'order_id'], values='revenue').groupby('day').agg({'revenue': 'mean'}).reset_index()
avg_revenue_pivot_by_weekday = df.pivot_table(index=['day_of_week', 'order_id'], values='revenue').groupby('day_of_week').agg({'revenue': 'mean'}).reindex(cats).reset_index()
avg_revenue_pivot_by_hour = df.pivot_table(index=['hour', 'order_id'], values='revenue').groupby('hour').agg({'revenue': 'mean'}).reset_index()

In [56]:
avg_revenue_params = {
    'data1': {'df': avg_revenue_pivot_by_month, 'name': 'По месяцам', 'x': 'month', 'y': 'revenue', 'hoverinfo': 'y', 'row': 1, 'col': 1},
    'data2': {'df': avg_revenue_pivot_by_day, 'name': 'По дням', 'x': 'day', 'y': 'revenue', 'hoverinfo': 'y', 'row': 1, 'col': 2},
    'data3': {'df': avg_revenue_pivot_by_weekday, 'name': 'По дням недели', 'x': 'day_of_week', 'y': 'revenue', 'hoverinfo': 'y', 'row': 2, 'col': 1},
    'data4': {'df': avg_revenue_pivot_by_hour, 'name': 'По часам', 'x': 'hour', 'y': 'revenue', 'hoverinfo': 'y', 'row': 2, 'col': 2},
}

In [57]:
fig = plot_bar_nm(2, 2, avg_revenue_params)

fig.update_xaxes(tickangle=45, tickvals = avg_revenue_pivot_by_month['month'], showgrid=False, row=1, col=1)
fig.update_xaxes(tickangle=45, tickvals = avg_revenue_pivot_by_day['day'], showgrid=False, row=1, col=2)
fig.update_xaxes(showgrid=False, row=2, col=1)
fig.update_xaxes(tickangle=45, tickvals = avg_revenue_pivot_by_hour['hour'], showgrid=False, row=2, col=2)

fig.update_layout(title_text='Средний чек по времени', height=800)
fig.show()

- В весенние месяцы покупатели в среднем делают более дешевые покупки;
- Самый выгодный месяц январь, скорее всего связано с большим количеством выходных дней;
- Средний чек во вторник и пятницу больше чем в остальные дни;
- В 7 утра самый большой чек, возможно это юр.лица закупаются перед началом рабочего дня.

### Исследование товарного ассортимента

**Топ-10 по количеству проданных товаров:**

In [58]:
product_quantity = df.groupby('product').agg({'quantity': 'sum', 'revenue': 'sum', 'category': 'first'}).sort_values(by='quantity', ascending=False)

In [59]:
product_quantity.head(10)

,quantity,revenue,category
product,,,
Муляж ЯБЛОКО 9 см красное,618,32702,Интереьер
Вешалки мягкие для деликатных вещей 3 шт шоколад,335,49596,Товары для дома
Муляж Яблоко зеленый 9 см полиуретан,188,10492,Интереьер
"Крепеж для пружины дверной, 1107055",170,3290,Все для ремонта
Плечики пластмассовые Размер 52 - 54 Тула 1205158,160,3210,Товары для дома
Муляж Банан желтый 21 см полиуретан,109,5831,Интереьер
"Щетка-сметка 4-х рядная деревянная 300 мм (фигурная ручка) ворс 5,5 см 1801096",105,6810,Все для уборки
"Ёрш унитазный с деревянной ручкой , Ваир 1712012",103,5633,Все для уборки
"Стяжка оконная с болтом СТ-55 цинк, 1108354",100,1944,Все для ремонта


*Самый продаваемый товар "Муляж ЯБЛОКО 9 см красное"*

**Худшие 10 товаров по продажам:**

In [60]:
product_quantity.tail(10)

,quantity,revenue,category
product,,,
Скатерть 350х150 см WELLNESS MT355-Джулия*16 36% полиэстер 64% хлопок,1,2249,Кухонные принадлежности
Корзина для белья INFINITY CURVER 59 л белая 04754-N23-00,1,1087,Товары для дома
Скатерть 150х120 см WELLNESS WT125-Куэна 100% полиэстер,1,764,Кухонные принадлежности
"Скалка силиконовая, Lekue, 0200800M02U050",1,1312,Кухонные принадлежности
Сито WEBBER из нержавеющей стали d = 21 см с пластиковой ручкой ВЕ-7335,1,194,Кухонные принадлежности
Синнингия (глоксиния) фиолетовая d-12 см h-20,1,389,Сад и огород
Сиденье для унитаза Росспласт АЖУР белое РП-813,1,374,Товары для дома
"Сиденье для гладильной доски Leifheit Niveau, 71325",1,6149,Товары для дома
Сиденье для ванны пластмассовое М1552 1714003,1,517,Товары для дома


*Дорогая термокружк не пользуется популярностью.*

**Топ-10 товаров по выручке:**

In [61]:
product_revenue = df.groupby('product').agg({'quantity': 'sum', 'revenue': 'sum', 'category': 'first'}).sort_values(by='revenue', ascending=False)

In [62]:
product_revenue.head(10)

,quantity,revenue,category
product,,,
Простынь вафельная 200х180 см WELLNESS RW180-01 100% хлопок,30,53232,Товары для дома
Сумка-тележка 2-х колесная Gimi Argo синяя,47,50405,Товары для дома
Вешалки мягкие для деликатных вещей 3 шт шоколад,335,49596,Товары для дома
Тележка багажная DELTA ТБР-22 синий грузоподъемность 20 кг сумка и 50 кг каркас РОССИЯ,59,33992,Товары для дома
Муляж ЯБЛОКО 9 см красное,618,32702,Интереьер
"Сумка-тележка хозяйственная Andersen Scala Shopper Plus, Lini, синяя 133-108-90",5,28045,Товары для дома
"Урна уличная ""Гео"", Hobbyka/Хоббика, 59*37,5см, сталь",5,24370,Товары для дома
"Веник сорго с деревянной ручкой с 4-мя швами, Rozenbal, R206204",37,20010,Все для уборки
Сумка-тележка 3-х колесная Gimi Tris Floral синяя,7,18893,Товары для дома


*Больше всего выручки получено на продаже "Простынь вафельная 200х180 см WELLNESS RW180-01 100% хлопок"*

**Худшие 10 товаров по выручке:**

In [63]:
product_revenue.tail(10)

,quantity,revenue,category
product,,,
"Бархатцы Веселая полянка 0,3 г 4660010777505",1,11,Сад и огород
"Календула Суприм 0,5 г 4650091480227",1,11,Сад и огород
"Огурец Засолочный 0,3 г 4660010776102",1,10,Сад и огород
Петрушка Итальянский гигант 2 г 4660010776553,1,10,Сад и огород
"Незабудка смесь 0,1 г 4650091480340",1,10,Сад и огород
"Цинния Оранжевый король 0,5 г 4660010770520",1,10,Сад и огород
"Цинния Коралловая красавица 0,2 г 4660010773323",1,10,Сад и огород
Морковь Детская сладость 2 г 4660010775921,1,10,Сад и огород
"Горох Амброзия 10,0 г 4660010772616",1,9,Сад и огород


*Тут не все так однозначно, многие семена стоят дешего, это не значит что они чем-то хуже других товаров.*

**Посмотрим на распределение продажи товара по категориям:**

In [64]:
quantity_by_category = (df
                        .groupby('category')
                        .agg({'quantity': 'sum'})
                        .sort_values(by='quantity', ascending=False)
                        .reset_index()
                        )

In [65]:
revenue_by_category = (df
                        .groupby('category')
                        .agg({'revenue': 'sum'})
                        .sort_values(by='revenue', ascending=False)
                        .reset_index()
                        )

In [66]:
fig1 = px.bar(quantity_by_category,  
              x='category', 
              y='quantity',
              labels={'category': 'Категория', 'quantity': 'Количество'}
              )
fig2 = px.bar(revenue_by_category,  
              x='category', 
              y='revenue',
              labels={'category': 'Категория', 'revenue': 'Выручка'}
              )

fig = make_subplots(rows=1, 
                    cols=2, 
                    subplot_titles=("Количество проданного товара по категориям", "Размер выручки по категориям")
                    )

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)

fig.update_layout(height=700)
fig.show()

- Больше всего товаров продается в категории "Сад и огород";
- Самую большую выручку приносит категория "Товары для дома".

**Посмотрим на количество уникальных товаров по категориям:**

In [67]:
df.groupby('category').agg({'product': 'nunique'}).sort_values(by='product', ascending=False)

,product
category,
Сад и огород,941
Товары для дома,703
Кухонные принадлежности,319
Интереьер,181
Все для уборки,91
Все для ремонта,89


*Самый большой ассортимент в категории "Сад и огород", дальше идут "Товары для дома".*

### Анализ основного и дополнительного товара

**Создадим класс и метод в нем, который будет советовать дополнительный товар, на основе товара в заказе:**

In [68]:
class AdditionalProduct:
    """Класс для рекомендации дополнительного товара."""

    def __init__(self, data, category_col, rate=0.5):
        """
        Метод инициализации класса AdditionalProduct.

        data - датафрейм
        category_col - название столбца с категориями товара
        rate - как часто товар покупали, как дополнительный
        """

        self.data = data
        self.category_col = category_col
        self.rate = rate
        self.__suggest_dfs = self.__categorize()

    def __categorize(self):
        """Метод для составления словаря датафреймов по категориям."""

        dfs_dict = {}   # Словарь: ключ - название категории, значение - датафрейм

        for x in self.data[self.category_col].unique():
            suggest_df_by_category = (self.data[self.data[self.category_col] == x]
                                        .groupby('product')
                                        .agg({'main': 'sum', 'additional': 'sum'})
                                        .reset_index()
                                     )
            suggest_df_by_category['additional_rate'] = (
                    suggest_df_by_category['additional'] / 
                    (suggest_df_by_category['additional'] + suggest_df_by_category['main'])
                    )
            suggest_df_by_category = (
                suggest_df_by_category[suggest_df_by_category['additional_rate'] >= self.rate] 
            )

            dfs_dict[x] = suggest_df_by_category[['product', 'additional_rate']]
        
        return dfs_dict

    def get_additional_product(self, row):
        """
        Метод для рекомендации дополнительного товара.

        row - строка содержащая товар для которого нужно получить рекомендацию
        """

        category = row['category']

        df_by_category = self.__suggest_dfs[category]      

        n = 0
        suggest = df_by_category['product'].sample(replace=False)
        while n < 10:
            if len(self.data[self.data['product'].isin(suggest)]) < 3:
                n += 1
            else: 
                return suggest.values[0]

        return suggest.values[0]

In [69]:
ap = AdditionalProduct(df, 'category')

In [70]:
df['additional_product'] = df.apply(ap.get_additional_product, axis=1)

In [71]:
df.sample(5)

,date,customer_id,order_id,product,quantity,price,revenue,hour,day,day_of_week,week,month,year,dt,unique_products_count,main,additional,category,additional_product
2629,2019-04-03 16:00:00,b7b865ab-0735-407f-8d0c-31f74d2806cc,14611,Рассада томата (помидор) Ола Полка № 96 сорт детерминантный раннеспелый оранжевый,5,38,190,16,3,Wednesday,14,2019-04-01,2019,2019-04-03,4,0,1,Сад и огород,Рассада Дыни сорт Дина горшок 9х9 см P-9
5689,2019-07-05 14:00:00,4a3e8c01-1d47-4867-8a7f-14195a8dbb3d,14893,Базилик овощной Тонус в кассете 4 штуки среднеспелый,1,60,60,14,5,Friday,27,2019-07-01,2019,2019-07-05,24,0,1,Сад и огород,Роза кордана Красная Мерседес d-10 см
918,2018-11-22 22:00:00,138739ef-7689-4359-952c-ef623835db85,69089,Коврик придверный APACHE 45х76 см Flagstone 5415,1,1199,1199,22,22,Thursday,47,2018-11-01,2018,2018-11-22,1,1,0,Товары для дома,Сушилка для белья ЛИАНА ЛЮКС 190 см потолочная
5533,2019-06-22 06:00:00,1d6c8c1f-a799-4418-9af2-1ded47d7a85c,14833,"Рассада Кабачка сорт Аэронавт, кассета по 6шт",1,120,120,6,22,Saturday,25,2019-06-01,2019,2019-06-22,12,0,1,Сад и огород,Энотера Миссурийская Золотая желтый объем 1 л
2969,2019-04-19 18:00:00,498f12a4-6a62-4725-8516-cf5dc9ab8a3a,71204,Салфетка Protec Textil Polyline 30х43 см Аметист белая 6230,60,191,11460,18,19,Friday,16,2019-04-01,2019,2019-04-19,1,1,0,Все для уборки,"Швабра САЛЬСА треугольная МИНИ, Y8110"


### Вывод

**По результатам проверки описательной статистики:**

- Среднее количество товаров в заказе 2.8;
- Был очень крупный заказ на 1000 вантузов, отнесли его к аномалии;
- Большой разброс цен, стандартное отклонение 945;
- Поровну распределились основные и дополнительные товары.

**По анализу количества заказов по времени и сезону:**

- Весной, ожидаемо, товары категории "Сад и огород" продаются лучше, чем в остальные сезоны;
- Во все остальные сезоны лидирует категория "Товары для дома";
- По дням картина плюс минус одинаковая;
- Больше всего заказов делают в будние дни, меньше всего по выходным;
- Пик заказов приходится на промежуток с 10 до 13 часов.

**По анализу количества покупателей по времени картина аналогична количеству заказов.**

**По анализу выручки по времени:**

- Был выброс 18 июня 2019 в 15 часов (вторник) - это аномалия с "вантузами";
- Самую большую выручку магазин сделал в последние месяцы 2018 года;
- Во вторник выручка больше, чем в другие дни;
- В течении суток картина аналогична количеству заказов и оснавная выручка делается днем.

**По анализу среднего чека по времени:**

- В весенние месяцы покупатели, в среднем, делают более дешевые покупки;
- Самый выгодный месяц январь, скорее всего, это связано с большим количеством выходных дней;
- Средний чек во вторник и пятницу больше, чем в остальные дни;
- В 7 утра самый большой чек, возможно это юр.лица закупаются перед началом рабочего дня.

**По исследованию товарного ассортимента:**

- Самым продаваемым товаром оказался "Муляж ЯБЛОКО 9 см красное" из категории "Интерьер", его купили 618 штук;
- Хуже всего продавалась "Термокружка";
- Самую большую выручку сделал товар "Простынь вафельная 200х180 см WELLNESS RW180-01 100% хлопок", он принес 53 тыс у.е.;
- Больше всего товаров продается в категории "Сад и огород";
- Самую большую выручку приносит категория "Товары для дома";
- Самый большой ассортимент в категории "Сад и огород", дальше идут "Товары для дома";

**Так же хотелось бы отметить, что был создан класс для рекомендации дополнительного товара на основе добавленного в заказ.**

## Проверка статистических гипотез

**На основании исследовательского анализа проверим следующие гипотезы:**

- Средний чек по будням такой же, как и по выходным;
- Средний чек днем такой же, как и ночью;
- Средний чек в разные сезоны не отличается. Мы располагаем данными с 1 октября 2018 по 31 октября 2019, 
  поэтому для проверки будут взяты только данные с 1 декабря 2018 по 31 августа 2019 (зима, весна, лето);
- Для всех гипотез возьмем критический уровень статистической значимости 5%.

### Средний чек по будням такой же, как и по выходным

***Сформулируем нулевую гипотезу:*** *средний чек по будням такой же, как и по выходным.*  
***Сформулируем алтернативную гипотезу:*** *средний чек по будням и по выходным различаются.*

**Создадим две выборки weekday и day_off:**

**Посмотрим на распределение по выборкам:**

In [72]:
def draw_distribution(samples, titles):
    """
    Функция для отрисовки распределения по выборкам.   

    samples - кортеж содержащий выборки
    titles - кортеж содержащий заголовки для выборок
    """

    fig = make_subplots(rows=2, 
                        cols=len(samples), 
                        horizontal_spacing = 0.05, 
                        vertical_spacing = 0.1, 
                        subplot_titles=titles
                       )

    for n, value in enumerate(samples):
        fig.add_trace(go.Histogram(
            x=value,
            showlegend=False,
            hoverinfo="x"),
            row=1, col=n+1
        )

        fig.add_trace(go.Box(
            x=value,
            showlegend=False,
            name='',
            hoverinfo="x"),
            row=2, col=n+1
        )

    fig.update_layout(height=800)
    fig.show()

In [73]:
day_off_list = ['Saturday', 'Sunday']

In [74]:
weekday = df[~df['day_of_week'].isin(day_off_list)].groupby('order_id')['revenue'].sum().reset_index()['revenue']

In [75]:
day_off = df[df['day_of_week'].isin(day_off_list)].groupby('order_id')['revenue'].sum().reset_index()['revenue']

In [76]:
draw_distribution((weekday, day_off), ('"Будние"', '"Выходные"'))

*Много выбросов, будем использовать непараметрический тест Манна-Уитни.*

In [77]:
def mannwhitneyu(sample1, sample2, alternative='two-sided', alpha=.05):
    """
    Функция для проверки статистической гипотезы с помощью U-критерия Манна—Уитни.

    sample1 - первая выборка
    sample2 - вторая выборка
    alternative - вид проверки {two-sided, less, greater}
    alpha - критический уровень статистической значимости       
    """
    
    alpha = alpha 
        
    results = st.mannwhitneyu(sample1, sample2, alternative=alternative)
    print('p-значение:', results.pvalue)
    
    if results.pvalue < alpha:
        print("Отвергаем нулевую гипотезу")
    else:
        print("Не получилось отвергнуть нулевую гипотезу") 

In [78]:
mannwhitneyu(weekday, day_off)

p-значение: 0.13933503131871902
Не получилось отвергнуть нулевую гипотезу


*После проведения теста, можно оставить нулевую гипотезу, средний чек по будням не отличается.*

### Средний чек днем такой же, как и ночью

***Сформулируем нулевую гипотезу:*** *средний чек днем такой же, как и ночью.*  
***Сформулируем алтернативную гипотезу:*** *средний чек днем и ночью различаются.*

**Создадим две выборки day и night:**

**Посмотрим на распределение по выборкам:**

In [79]:
day = df[(df['hour'] >= 6) & (df['hour'] <= 23)].groupby('order_id')['revenue'].sum().reset_index()['revenue']

In [80]:
night = df[(df['hour'] >= 0) & (df['hour'] < 6)].groupby('order_id')['revenue'].sum().reset_index()['revenue']

In [81]:
draw_distribution((day, night), ('"День"', '"Ночь"'))

*Много выбросов, будем использовать непараметрический тест Манна-Уитни.*

In [82]:
mannwhitneyu(day, night)

p-значение: 0.7580129831918525
Не получилось отвергнуть нулевую гипотезу


*После проведения теста, можно оставить нулевую гипотезу, средний чек днем такой же, как и ночью.*

### Средний чек в разные сезоны не отличается

***Сформулируем нулевую гипотезу:*** *средний чек в разные сезоны не отличается.*  
***Сформулируем алтернативную гипотезу:*** *средний чек в разные сезоны различается.*

**Создадим три выборки winter, spring, summer:**

In [83]:
winter_sample = df[df['month'].isin(['2018-12-01', '2019-02-01', '2019-03-01'])].groupby('order_id')['revenue'].sum().reset_index()['revenue']

In [84]:
spring_sample = df[df['month'].isin(['2019-03-01', '2019-04-01', '2019-05-01'])].groupby('order_id')['revenue'].sum().reset_index()['revenue']

In [85]:
summer_sample = df[df['month'].isin(['2018-06-01', '2019-07-01', '2019-08-01'])].groupby('order_id')['revenue'].sum().reset_index()['revenue']

**Посмотрим на распределение по выборкам:**

In [86]:
draw_distribution((winter_sample, 
                   spring_sample, 
                   summer_sample), 
                  ('"Зима"', 
                   '"Весна"', 
                   '"Лето"')
)

*Много выбросов, будем использовать непараметрический тест Манна-Уитни.*

**Для начала сравним выборки winter и spring:**

In [87]:
mannwhitneyu(winter_sample, spring_sample)

p-значение: 0.6677191854511851
Не получилось отвергнуть нулевую гипотезу


**Теперь выборки winter и summer:**

In [88]:
mannwhitneyu(winter_sample, summer_sample)

p-значение: 0.6483843232002131
Не получилось отвергнуть нулевую гипотезу


**Теперь выборки spring и summer:**

In [89]:
mannwhitneyu(spring_sample, summer_sample)

p-значение: 0.441026550690205
Не получилось отвергнуть нулевую гипотезу


*После проведения тестов не получилось отвергнуть нулевую гипотезу, разницы в среднем чеке по сезонам нет.*

### Вывод

**Были проверены следующие статистические гипотезы:**

- Средний чек по будням такой же, как и по выходным;
- Средний чек днем такой же, как и ночью;
- Средний чек в разные сезоны не отличается. Мы располагаем данными с 1 октября 2018 по 31 октября 2019, поэтому для проверки будут взяты только данные с 1 декабря 2018 по 31 августа 2019 (зима, весна, лето);

**Для всех гипотез был взят критический уровень статистической значимости в 5%. Все тесты показали, что нулевая гипотеза верна, т.к. было множественное сравнение и если бы хоть одна гипотеза показала статистически значимый результат, необходимо было бы применить поправку Бонферрони, в таком случае критический уровень статистической значимости составил бы 1%.**

## Расчет бизнес-показателей

**Для расчета бизнес-показателей выберем следующие метрики:**
- DAU (по месяцам)
- WAU (по месяцам)
- MAU (по месяцам)
- AOV - средний чек (по месяцам)

In [90]:
DAU = (df
        .groupby(['month', 'dt'])
        .agg({'customer_id': 'nunique'})
        .groupby('month')['customer_id']
        .mean()
        .reset_index()
        .rename(columns={'customer_id': 'DAU'})
)

In [91]:
WAU = (df
        .groupby(['month', 'week'])
        .agg({'customer_id': 'nunique'})
        .groupby('month')['customer_id']
        .mean()
        .reset_index()
        .rename(columns={'customer_id': 'WAU'})
)

In [92]:
MAU = (df
        .groupby('month')
        .agg({'customer_id': 'nunique'})
        .reset_index()
        .rename(columns={'customer_id': 'MAU'})
)

In [93]:
AOV = avg_revenue_pivot_by_month.rename(columns={'revenue': 'AOV'})

In [94]:
kpi = DAU.merge(WAU).merge(MAU).merge(AOV)

In [95]:
kpi_params = {
    'data1': {'df': kpi, 'name': 'DAU', 'x': 'month', 'y': 'DAU', 'hoverinfo': 'y', 'row': 1, 'col': 1},
    'data2': {'df': kpi, 'name': 'WAU', 'x': 'month', 'y': 'WAU', 'hoverinfo': 'y', 'row': 1, 'col': 2},
    'data3': {'df': kpi, 'name': 'MAU', 'x': 'month', 'y': 'MAU', 'hoverinfo': 'y', 'row': 2, 'col': 1},
    'data4': {'df': kpi, 'name': 'AOV', 'x': 'month', 'y': 'AOV', 'hoverinfo': 'y', 'row': 2, 'col': 2},
}

In [96]:
fig = plot_bar_nm(2, 2, kpi_params)

fig.update_xaxes(tickangle=45, tickvals = kpi['month'], showgrid=False, row=1, col=1)
fig.update_xaxes(tickangle=45, tickvals = kpi['month'], showgrid=False, row=1, col=2)
fig.update_xaxes(tickangle=45, tickvals = kpi['month'], showgrid=False, row=2, col=1)
fig.update_xaxes(tickangle=45, tickvals = kpi['month'], showgrid=False, row=2, col=2)

fig.update_layout(title_text='KPI интернет-магазина', height=800)
fig.show()

- Метрики активных пользователей довольно стабильны;
- Есть прирост в феврале, марте, апреле 2019, скорее всего связано с закупом в категории "Сад и огород";
- При этом средний чек в эти месяцы минимален.

## Общий вывод

**В результате анализа было выяснено:**

- Среднее количество товаров в заказе 2.8;
- Основной ассортимент товаров в магазине представлен категорией "Сад и огород";
- Товаров, которые покупают как отдельную единицу и тех, что предпочитают как дополнительный, в товарном ассортименте поровну;
- Весной количество товаров категории "Сад и огород" продается больше, чем в остальные сезоны;
- Во все сезоны, кроме весны, по количеству проданных товаров, лидирует категория "Товары для дома";
- Больше всего заказов делается в будние дни;
- Пик заказов приходится на промежуток с 10 до 13 часов;
- Самую большую выручку магазин сделал в последние месяцы 2018 года;
- Самый большой средний чек достигнут в январе;
- Самым продаваемым товаром оказался товар из категории "Интерьер";
- Самые дешевые товары, которые покупают по одной штуке, в категории "Сад и огород";

**Были проверены статистические гипотезы:**

- Средний чек по будням такой же, как и по выходным;
- Средний чек днем такой же, как и ночью;
- Средний чек в разные сезоны не отличается. Мы располагаем данными с 1 октября 2018 по 31 октября 2019, поэтому для проверки будут взяты только данные с 1 декабря 2018 по 31 августа 2019 (зима, весна, лето).

**Ни один тест не показал статистически значимого результата. Средний чек в разные временные промежутки не отличается.**

**Были расчитаны бизнес-показатели:**

- DAU (по месяцам)
- WAU (по месяцам)
- MAU (по месяцам)
- AOV - средний чек (по месяцам)

**На основе них было выяснено, что количество активных пользователей стабильно, но есть прирост в феврале, марте, апреле 2019 года. Средний чек, меньше всего в весенние месяцы.**

*Хотелось бы отметить, что был создан класс и метод в нем, для рекомендации дополнительного товара, но основе товаров из заказа.*

**Рекомендации:** 

- Расширить ассортимент в категории "Товары для дома", хоть и по количеству категория "Сад и огород" лидирует, но именно "Товары для дома" делают основную выручку, сопоставимую с совокупной суммой продаж в других категориях;
- Стимулировать покупателей скидками и акциями, чтобы они делали заказы в выходные и в ночное время, увеличив количество заказов в это время, увеличим и выручку;
- Делать основные закупки семян и рассады к весне, в остальное время они не пользуются популярностью;
- В августе и сентябре, традиционно сезон отпусков, предложить скидки в эти месяцы, тем самым увеличим количество заказов.